In [ ]:
import requests, time, json
import pandas as pd

# Base Merimée

JUST DOWNLOAD the parquet file from here: https://data.culture.gouv.fr/explore/dataset/liste-des-immeubles-proteges-au-titre-des-monuments-historiques/export/?disjunctive.departement_en_lettres

The API seems to retrieve less columns, and it's hard to retrive all lines due to limits + data quality issues, that make it hard to filter.

---

I leave this for reference:

In [ ]:
def fetch_batch(base, departement, limit=100, offset=0):
    url = f"{base}?where=departement_format_numerique%3D{departement}&limit={limit}&offset={offset}"
    print(f"→ GET {url}")
    resp = requests.get(url)
    resp.raise_for_status()
    return resp.text


def parse_json(text):
    return json.loads(text)


def to_dataframe(records):
    return pd.json_normalize(records)


def fetch_all(base, departements, output_filename="merimee_full.parquet"):
    limit = 100
    all_results = []

    for departement in departements:
        print(f"\n Departement: {departement}")
        offset = 0

        while True:
            print(f"  Fetching offset {offset}")
            text = fetch_batch(base, departement, limit=limit, offset=offset)
            data = parse_json(text)
            results = data.get("results", [])

            if not results:
                print(f"{departement} ends")
                break

            all_results.extend(results)
            offset += limit
            time.sleep(0.2)

    df = to_dataframe(all_results)
    df.to_parquet(output_filename, index=False)

In [ ]:
# List of all French department codes as strings
departements = [
    "01",
    "02",
    "03",
    "04",
    "05",
    "06",
    "07",
    "08",
    "09",
    "10",
    "11",
    "12",
    "13",
    "14",
    "15",
    "16",
    "17",
    "18",
    "19",
    "'2A'",
    "'2B'",  # Corsica needs those quotes because they aren't real numbers
    "21",
    "22",
    "23",
    "24",
    "25",
    "26",
    "27",
    "28",
    "29",
    "30",
    "31",
    "32",
    "33",
    "34",
    "35",
    "36",
    "37",
    "38",
    "39",
    "40",
    "41",
    "42",
    "43",
    "44",
    "45",
    "46",
    "47",
    "48",
    "49",
    "50",
    "51",
    "52",
    "53",
    "54",
    "55",
    "56",
    "57",
    "58",
    "59",
    "60",
    "61",
    "62",
    "63",
    "64",
    "65",
    "66",
    "67",
    "68",
    "69",
    "70",
    "71",
    "72",
    "73",
    "74",
    "75",
    "76",
    "77",
    "78",
    "79",
    "80",
    "81",
    "82",
    "83",
    "84",
    "85",
    "86",
    "87",
    "88",
    "89",
    "90",
    "91",
    "92",
    "93",
    "94",
    "95",
    "971",
    "972",
    "973",
    "974",
    "976",
]

base_url = (
    "https://data.culture.gouv.fr/api/explore/v2.1/catalog/datasets/"
    "liste-des-immeubles-proteges-au-titre-des-monuments-historiques/records"
)
fetch_all(base_url, departements, output_filename="merimee_full.parquet")